In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')
#if you do not have access to the twenty qubit machine, use this publically available ibmqx2 instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the ibmqx2 instead
device = provider.get_backend('ibmqx2')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 2
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    description: 5 qubit device
    memory: True
    online_date: 2017-01-24T05:00:00+00:00
    max_experiments: 75
    conditional: False
    max_shots: 8192
    sample_name: sparrow
    allow_object_storage: True
    quantum_volume: 8
    url: None
    n_registers: 1
    allow_q_object: True
    backend_name: ibmqx2
    allow_q_circuit: False
    open_pulse: False
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]
    credits_required: True

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 48.9011 µs / 47.74382 µs / 0.0 / 0.00068 / 0.00136 / 0.015
    Q1 / 5.23787 GHz / 75.88025 µ

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
transQcList = []
pi = math.pi

In [6]:
import numpy as np
def input_state(circ, nqubits, value):
    for j in range(nqubits):
        circ.h(j)
        circ.u1(-value * math.pi/float(2**(j)), j)
def qft(circ, q, n):
    """n-qubit QFT on q in circ."""
    for j in range(n):
        for k in range(j):
            circ.cu1(np.pi / float(2**(j - k)), q[j], q[k])
        circ.h(q[j])
        
def qpe(value):
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    input_state(circuit, 4, value)


    qft(circuit, q, 4)

    for k in range(4):
        circuit.measure(q[k],c[k])

    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    #print("QPE circuit size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), " CNOT =",num_cnots, "subcircuits =",trans_qc.num_tensor_factors())
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

def qpe_assertion(value):
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    input_state(circuit, 4, value)
        

    qft(circuit, q, 4)
    qubitList = [q[3]]
    
    assertion_value = int("{0:04b}".format(value)[0])
    #print("{0:04b}".format(value))
    #print(assertion_value)
    classical_assertion(circuit, qubitList, assertion_value)

    for k in range(4):
        circuit.measure(q[k],c[k])

        
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    #print("QPE circuit with assertion size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "CNOT =", num_cnots , "subcircuits =",trans_qc.num_tensor_factors())
    return trans_qc, num_cnots

In [7]:
def qpeExperiment(value):
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    for i in range(10):
        trans_qc, num_cnots = qpe(value)
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = qpe(value)
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit without assertion: ", num_cnots)
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = qpe_assertion(value)
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = qpe_assertion(value)
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            transQcList.append(trans_qc)
            break


In [8]:
qpeExperiment(0)
qpeExperiment(1)
qpeExperiment(8)
qpeExperiment(3)
qpeExperiment(12)
qpeExperiment(7)
qpeExperiment(14)
qpeExperiment(15)

number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  20
number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17


In [9]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of output state |0000> without assertion

In [10]:
calcSuccessrate(res.get_counts(0), ["0000"], 0)

total_count =  8192 success_count =  2250 success_rate =  27.4658203125 %


Success rate of output state |0000> with assertion

In [11]:
calcSuccessrate(res.get_counts(1), ["0 0000"], 1)

total_count =  5426 success_count =  3136 success_rate =  57.795798009583486 %


Success rate of output state |0001> without assertion

In [12]:
calcSuccessrate(res.get_counts(2), ["0001"], 0)

total_count =  8192 success_count =  1924 success_rate =  23.486328125 %


Success rate of output state |0001> with assertion

In [13]:
calcSuccessrate(res.get_counts(3), ["0 0001"], 1)

total_count =  5473 success_count =  2527 success_rate =  46.172117668554726 %


Success rate of output state |1000> without assertion

In [14]:
calcSuccessrate(res.get_counts(4), ["1000"], 0)

total_count =  8192 success_count =  1725 success_rate =  21.05712890625 %


Success rate of output state |1000> with assertion

In [15]:
calcSuccessrate(res.get_counts(5), ["0 1000"], 1)

total_count =  5541 success_count =  2482 success_rate =  44.79335859953077 %


Success rate of output state |0011> without assertion

In [16]:
calcSuccessrate(res.get_counts(6), ["0011"], 0)

total_count =  8192 success_count =  1907 success_rate =  23.27880859375 %


Success rate of output state |0011> with assertion

In [17]:
calcSuccessrate(res.get_counts(7), ["0 0011"], 1)

total_count =  5603 success_count =  2085 success_rate =  37.21220774585044 %


Success rate of output state |1100> without assertion

In [18]:
calcSuccessrate(res.get_counts(8), ["1100"], 0)

total_count =  8192 success_count =  2210 success_rate =  26.9775390625 %


Success rate of output state |1100> with assertion

In [19]:
calcSuccessrate(res.get_counts(9), ["0 1100"], 1)

total_count =  5986 success_count =  1910 success_rate =  31.907784831272973 %


Success rate of output state |0111> without assertion

In [20]:
calcSuccessrate(res.get_counts(10), ["0111"], 0)

total_count =  8192 success_count =  1490 success_rate =  18.1884765625 %


Success rate of output state |0111> with assertion

In [21]:
calcSuccessrate(res.get_counts(11), ["0 0111"], 1)

total_count =  5717 success_count =  1863 success_rate =  32.58702116494665 %


Success rate of output state |1110> without assertion

In [22]:
calcSuccessrate(res.get_counts(12), ["1110"], 0)

total_count =  8192 success_count =  1620 success_rate =  19.775390625 %


Success rate of output state |1110> with assertion

In [23]:
calcSuccessrate(res.get_counts(13), ["0 1110"], 1)

total_count =  4524 success_count =  1360 success_rate =  30.061892130857647 %


Success rate of output state |1111> without assertion

In [24]:
calcSuccessrate(res.get_counts(14), ["1111"], 0)

total_count =  8192 success_count =  1584 success_rate =  19.3359375 %


Success rate of output state |1111> with assertion

In [25]:
calcSuccessrate(res.get_counts(15), ["0 1111"], 1)

total_count =  5639 success_count =  1566 success_rate =  27.770881361943605 %
